In [3]:
from pathlib import Path
from PIL import Image
import numpy as np
import imageio.v2 as imageio  # imageio.v2 pour compatibilité

# ==========================
# Config
# ==========================
CANDIDATE_DIRS = [
    Path("/workspace/samples/06b_cgan_ia"),
    Path("workspace/samples/06b_cgan_ia"),
]

TARGET_DURATION_S = 20       # durée cible de la vidéo
TRANSITION_FRAMES = 2        # nombre de frames pour le crossfade
HOLD_FRAMES = 1              # frames "fixes" par image
OUT_MP4 = "cgan_multiclass_evolution.mp4"

MAX_WIDTH = 512              # pour compresser plus : réduire la résolution
MAX_HEIGHT = 512


def find_samples_dir():
    for d in CANDIDATE_DIRS:
        if d.exists():
            return d
    raise FileNotFoundError(f"Aucun dossier trouvé parmi : {CANDIDATE_DIRS}")


def load_images(samples_dir):
    exts = (".png", ".jpg", ".jpeg")
    files = sorted([p for p in samples_dir.iterdir()
                    if p.is_file() and p.suffix.lower() in exts])
    if not files:
        raise RuntimeError(f"Aucune image trouvée dans {samples_dir}")
    print(f"{len(files)} images trouvées dans {samples_dir}")

    imgs = [Image.open(f).convert("RGB") for f in files]

    # Normalisation de la taille + éventuel downscale
    base_w, base_h = imgs[0].size
    scale = min(MAX_WIDTH / base_w, MAX_HEIGHT / base_h, 1.0)
    new_size = (int(base_w * scale), int(base_h * scale))

    imgs = [im.resize(new_size, Image.BILINEAR) for im in imgs]
    print(f"Taille des frames : {new_size}")
    return imgs


def build_frames_with_transitions(imgs):
    frames = []

    for i in range(len(imgs) - 1):
        img1 = imgs[i]
        img2 = imgs[i + 1]

        # frames fixes
        for _ in range(HOLD_FRAMES):
            frames.append(img1)

        # crossfade
        for t in range(1, TRANSITION_FRAMES + 1):
            alpha = t / (TRANSITION_FRAMES + 1)
            blended = Image.blend(img1, img2, alpha)
            frames.append(blended)

    # Dernière image un peu plus longtemps
    for _ in range(HOLD_FRAMES + TRANSITION_FRAMES // 2):
        frames.append(imgs[-1])

    print(f"Nombre total de frames (avec transitions) : {len(frames)}")
    return frames


def save_mp4(frames, out_path, target_duration_s=TARGET_DURATION_S, tail_seconds=3):
    """
    target_duration_s : durée totale approximative (hors 'pause' finale)
    tail_seconds      : durée pendant laquelle on reste sur la dernière image
    """
    # On sépare logique : frames "normales" vs frames de fin
    base_frames = frames[:-1]  # tout sauf la dernière
    last_frame = frames[-1]

    # FPS calculé sur la partie "animée"
    total_base_frames = len(base_frames)
    fps = max(int(total_base_frames / target_duration_s), 1)
    print(f"FPS ≈ {fps} pour ~{target_duration_s}s d'animation + {tail_seconds}s de pause finale")

    # On reconstruit la séquence vidéo :
    video_frames = [np.array(f) for f in base_frames]

    # Ajout des frames de pause sur la dernière image
    tail_frames = int(tail_seconds * fps)
    for _ in range(tail_frames):
        video_frames.append(np.array(last_frame))

    imageio.mimsave(
        out_path,
        video_frames,
        fps=fps,
        codec="libx264",
        quality=8,  # tu peux ajuster si besoin
    )
    total_duration = len(video_frames) / fps
    print(f"Vidéo MP4 sauvegardée → {out_path} (durée ~{total_duration:.1f}s)")



if __name__ == "__main__":
    samples_dir = find_samples_dir()
    imgs = load_images(samples_dir)
    frames = build_frames_with_transitions(imgs)
    save_mp4(frames, OUT_MP4, target_duration_s=TARGET_DURATION_S, tail_seconds=1)


92 images trouvées dans /workspace/samples/06b_cgan_ia
Taille des frames : (512, 512)
Nombre total de frames (avec transitions) : 275
FPS ≈ 13 pour ~20s d'animation + 1s de pause finale
Vidéo MP4 sauvegardée → cgan_multiclass_evolution.mp4 (durée ~22.1s)
